In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from bigbird import modeling
from bigbird import utils
import tensorflow as tf
import numpy as np

In [3]:
import sentencepiece as spm

vocab = '/home/husein/b2b/sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s)
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [4]:
top_p = tf.placeholder(tf.float32, None, name = 'top_p')
temperature = tf.placeholder(tf.float32, None, name = 'temperature')

In [5]:
bert_config = {
    # transformer basic configs
    'attention_probs_dropout_prob': 0.1,
    'hidden_act': 'relu',
    'hidden_dropout_prob': 0.1,
    'hidden_size': 768,
    'initializer_range': 0.02,
    'intermediate_size': 3072,
    'max_position_embeddings': 4096,
    'max_encoder_length': 2048,
    'max_decoder_length': 1024,
    'num_attention_heads': 12,
    'num_hidden_layers': 12,
    'type_vocab_size': 2,
    'scope': 'pegasus',
    'use_bias': False,
    'rescale_embedding': True,
    'vocab_model_file': None,
    # sparse mask configs
    'attention_type': 'block_sparse',
    'norm_type': 'prenorm',
    'block_size': 64,
    'num_rand_blocks': 3,
    'vocab_size': 32000,
    'beam_size': 1,
    'alpha': 0.0,
    'couple_encoder_decoder': False,
    'num_warmup_steps': 10000,
    'learning_rate': 0.0001,
    'label_smoothing': 0.1,
    'optimizer': 'Adafactor',
    'use_tpu': True,
    'top_p': top_p,
    'temperature': temperature
}

In [6]:
model = modeling.TransformerModel(bert_config)

In [7]:
X = tf.placeholder(tf.int32, [None, None])

In [8]:
r = model(X, training = False)
r

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


((<tf.Tensor 'pegasus/log_probs:0' shape=(?, 1024) dtype=float32>,
  <tf.Tensor 'pegasus/logits:0' shape=(?, 1024, 32000) dtype=float32>,
  <tf.Tensor 'pegasus/while/Exit_1:0' shape=(?, 1024) dtype=int32>),
 <tf.Tensor 'pegasus/encoder/LayerNorm/batchnorm/add_1:0' shape=(?, 2048, 768) dtype=float32>)

In [9]:
logits = tf.identity(r[0][2], name = 'logits')
logits

<tf.Tensor 'logits:0' shape=(?, 1024) dtype=int32>

In [10]:
# files = tf.gfile.Glob('gs://mesolitica-tpu-general/t2t-summarization-v2/data/seq2*')

In [11]:
# batch_size = 4
# data_fields = {
#     'inputs': tf.VarLenFeature(tf.int64),
#     'targets': tf.VarLenFeature(tf.int64),
# }
# data_len = {
#     'inputs': 2048,
#     'targets': 1024,
# }

# def parse(serialized_example):

#     features = tf.parse_single_example(
#         serialized_example, features = data_fields
#     )
#     for k in features.keys():
#         features[k] = features[k].values
#         features[k] = tf.pad(
#             features[k], [[0, data_len[k] - tf.shape(features[k])[0]]]
#         )
#         features[k].set_shape((data_len[k]))

#     return features

# def _decode_record(example, name_to_features):
#     """Decodes a record to a TensorFlow example."""

#     # tf.Example only supports tf.int64, but the TPU only supports tf.int32.
#     # So cast all int64 to int32.
#     for name in list(example.keys()):
#         t = example[name]
#         if t.dtype == tf.int64:
#             t = tf.to_int32(t)
#         example[name] = t

#     return example

# d = tf.data.TFRecordDataset(files)
# d = d.map(parse, num_parallel_calls = 32)
# d = d.apply(
#     tf.contrib.data.map_and_batch(
#         lambda record: _decode_record(record, data_fields),
#         batch_size = batch_size,
#         num_parallel_batches = 4,
#         drop_remainder = True,
#     )
# )
# d = d.make_one_shot_iterator().get_next()
# d

In [12]:
import tensorflow as tf

ckpt_path = tf.train.latest_checkpoint('gs://mesolitica-tpu-general/bigbird-summarization-base')
ckpt_path

'gs://mesolitica-tpu-general/bigbird-summarization-base/model.ckpt-500000'

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [14]:
# r_ = sess.run(d)
# encoder.decode(r_['inputs'][2].tolist())
# encoder.decode(r_['targets'][2].tolist())

In [15]:
# import re
# import collections

# def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
#     """Compute the union of the current variables and checkpoint variables."""
#     assignment_map = {}
#     initialized_variable_names = {}

#     name_to_variable = collections.OrderedDict()
#     for var in tvars:
#         name = var.name
#         m = re.match('^(.*):\\d+$', name)
#         if m is not None:
#             name = m.group(1)
#         name_to_variable[name] = var

#     init_vars = tf.train.list_variables(init_checkpoint)
#     assignment_map = collections.OrderedDict()
#     for x in init_vars:
#         (name, var) = (x[0], x[1])

#         l = 'pegasus/' + name
#         l = l.replace('embeddings/weights', 'embeddings/word_embeddings')
#         l = l.replace('self/output', 'output')
#         l = l.replace('ffn/dense_1', 'output/dense')
#         l = l.replace('ffn', 'intermediate')
#         l = l.replace('memory_attention/output', 'attention/encdec_output')
#         l = l.replace('memory_attention', 'attention/encdec')

#         if l not in name_to_variable:
#             continue
#         assignment_map[name] = name_to_variable[l]
#         initialized_variable_names[l + ':0'] = 1

#     return (assignment_map, initialized_variable_names)

In [16]:
# t = tf.trainable_variables()
# assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(t, ckpt_path)

In [17]:
saver = tf.train.Saver()
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/bigbird-summarization-base/model.ckpt-500000


INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/bigbird-summarization-base/model.ckpt-500000


In [18]:
# var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
# saver = tf.train.Saver(var_list = var_lists)
# saver.restore(sess, 'gs://mesolitica-tpu-general/bigbird-summarization-small/model.ckpt-0')

In [19]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string.replace('\n', ' '))).strip()

In [20]:
string = """
Tidak ada apa-apa mengenai mesyuarat fakulti biologi Jumaat lalu di University of Alabama di Huntsville yang membayangkan pembunuhan yang akan berlaku, Profesor Debra Moriarity memberitahu wartawan Rabu. "Ia sebenarnya adalah satu-satunya mesyuarat fakulti yang sangat santai dan biasa," ahli biokim memberitahu CNN affiliate WAAY mengenai 13 orang yang duduk di sekitar meja bujur di Bilik 369 di Shelby Centre for Science and Technology. "Peristiwa akan datang, kelas penjadualan, belanjawan. Ia sebenarnya merupakan salah satu daripada mesyuarat fakulti yang paling mudah yang kami ada." Antara peserta adalah Amy Bishop, seorang ahli genetik yang terlatih Harvard dengan siapa Moriarity telah membangun hubungan profesional yang dipupuk oleh hakikat bahawa kedua-dua wanita bekerja dengan budaya sel. "Kadang-kadang anda meminjam sesuatu dari satu sama lain," katanya. "Kami telah bercakap mengenai cadangan pemberian cadangan bersama." Selepas kira-kira sejam, sebelum 4hb, Bishop - yang baru-baru ini dinafikan tempohnya - menamatkan tenang. "Secara tiba-tiba, dia hanya berdiri dan ditembak," kata Moriarity. Moriarity bertindak balas dengan cepat, menjatuhkan ke tangan dan lututnya di atas permaidani kelabu. "Hanya jatuh ke lantai dan merangkak di bawah meja dan merangkak ke arah Amy," kata Moriarity, yang memberi tumpuan kepada satu perkara. "Maksud saya, kamu merangkak di bawah meja, kamu melihat kaki orang yang menembak di atas meja, saya merangkul kakinya dan, saya tidak tahu apa yang saya fikirkan, saya tidak memikirkan apa-apa. hanya berfikir: \'Ambil dia!\' "Dan dia menghalang saya. Maksud saya, dia menarik kaki kakinya percuma dan saya berada di ambang pintu dengan belakang saya jenisnya. Dan saya fikir dia cuba menembak saya, tetapi ketika saya mula berteriak kepadanya, \'Amy, Amy, berfikir tentang cucu saya, berfikir tentang anak perempuan saya! Ini saya! Saya telah membantu anda sebelum ini; Saya akan membantu anda lagi! Jangan buat Amy ini! Jangan lakukan ini! \'"Uskup kemudian melangkah masuk ke dalam dewan, menunjuk pistol di Moriarity dan menarik pencetus, kata ahli biologi." Ia diklik, dan ia diklik lagi, dan saya merangkak kembali ke dalam bilik dan menutup pintu dan dia ditinggalkan di dalam dewan. "Orang-orang yang terselamat itu beraksi. Satu orang mengunci pintu kayu, satu lagi mengetuk meja ke atasnya, yang lain memindahkan peti sejuk ke tempatnya untuk menghalang pintu, yang lain dipanggil 911, yang lain berpindah ke Tiga orang yang maut, tiga lagi yang cedera, dua daripadanya kekal dirawat di rumah sakit pada hari Rabu dalam keadaan kritikal, menurut seorang jurubicara Hospital Huntsville, yang ketiga dilepaskan, Moriarity, yang menyertai fakulti sekolah pada tahun 1984, mengatakan bahawa mangsa tidak menjejaskan rancangannya untuk kekal di sekolah dan dia menolak apa-apa cadangan bahawa peranannya untuk mendapatkan Bishop di luar bilik itu adalah heroik. "Dia mengikuti saya di dewan dan kemudian senjata itu macet dan saya boleh mendapatkan ba ck di dalam bilik, "kata Moriarity. "Itu bukan seorang pahlawan. Itu hanya Tuhan yang melihat kamu." Dia berkata dia mempunyai sedikit masa untuk berfikir. "Dari awal hingga akhirnya kami mendapat sesuatu yang tidak dapat dibendung, ia tidak boleh melebihi 20 saat," katanya. Moriarity terus menolak cadangan bahawa apa-apa boleh dilakukan untuk melindungi mangsa. "Tidak ada cara untuk menjangka ini," katanya. "Dan tiada apa yang dapat dilakukan untuk menghentikannya, semuanya berlaku terlalu pantas." Dan dia bimbang bahawa apa-apa cubaan untuk mengetatkan keselamatan boleh membawa kesan negatif. "Ada kejahatan di dunia, malangnya orang baik disakiti oleh itu, tetapi universiti adalah tempat pemikiran bebas dan kebebasan untuk meneroka idea-idea dan mencari pengetahuan baru dan anda tidak mahu meletakkan sesuatu di tempat yang meredakannya. " Moriarity kembali ke pejabatnya pada hari Rabu dan berkata dia merancang untuk meneruskan pengajaran minggu depan. Dia meramalkan bahawa, dengan bantuan ubat anti-kecemasan, dia akan dapat tidur malam Rabu. "Saya telah bercakap dengan keluarga dan kawan-kawan dan hanya mendapatkan sokongan mereka membantu anda menanganinya," katanya. "Saya fikir sekarang kebanyakan dari kita mahu kembali ke sana dan mendapatkan sesuatu yang berlaku, membuat rancangan untuk siapa yang akan menutup kelas." Satu perkhidmatan peringatan untuk menghormati kehidupan orang mati - ahli fakulti Maria Davis, Adriel Johnson dan Gopi Podila - akan diadakan pada hari Jumaat.
"""

In [21]:
string2 = """
Gabungan parti Warisan, Pakatan Harapan, dan Upko hari ini mendedahkan calon-calon masing-masing untuk pilihan raya negeri Sabah, tetapi ketika pengumuman itu berlangsung, perwakilan PKR di dewan itu dilihat ‘gelisah’ seperti ‘tidak senang duduk’.

Sekumpulan anggota PKR kemudian dilihat meninggalkan dewan di Pusat Konvensyen Antarabangsa Sabah di Kota Kinabalu selepas berbincang dengan ketua PKR Sabah Christina Liew.

Semakan senarai-senarai calon berkenaan mendapati PKR hanya memperolehi separuh daripada jumlah kerusi yang diharapkan.

Semalam, PKR Sabah mengumumkan akan bertanding di 14 kerusi tetapi ketika Presiden Warisan Shafie Apdal mengumumkan calon gabungan tersebut hari ini, PKR hanya diberikan tujuh kerusi untuk bertanding.

Kerusi yang diberikan adalah Api-Api, Inanam, Tempasuk, Tamparuli, Matunggong, Klias, dan Sook.

Klias dan Sook adalah dua kerusi yang diberikan kepada PKR, sementara lima kerusi selebihnya pernah ditandingi oleh PKR pada pilihan raya umum 2018.

Dalam pengumuman PKR Sabah semalam, parti itu menjangkakan Warisan akan turut menyerahkan kerusi Kemabong, Membakut, dan Petagas kepada mereka.

Walau bagaimanapun, Warisan menyerahkan kerusi Kemabong kepada Upko dan mengekalkan bertanding untuk kerusi Membakut dan Petagas.

PKR juga menuntut empat daripada 13 kerusi baru yang diperkenalkan iaitu Segama, Limbahau, Sungai Manila, dan Pintasan tetapi Warisan membolot semua kerusi itu.

Sebagai pertukaran untuk kerusi yang diintainya, PKR bersedia untuk menyerahkan kerusi Kadaimaian, Kuala Penyu, dan Karanaan. Namun, ini dijangka tidak akan berlaku memandangkan parti tersebut tidak berpuas hati dengan agihan kerusi seperti yang diharapkan itu.

Selepas perwakilan dari PKR dan Liew keluar dari dewan tersebut, wartawan kemudian menyusuri Liew untuk mendapatkan penjelasannya.

Walau bagaimanapun, Liew enggan memberikan sebarang komen dan berkata bahawa dia ingin ke tandas.

Liew dan perwakilan PKR kemudian tidak kembali ke dalam dewan tersebut.

Apabila calon pilihan raya yang diumumkan diminta naik ke atas pentas untuk sesi bergambar, Liew tidak kelihatan.

Bilangan kerusi yang ditandingi oleh PKR kali ini hanya kurang satu kerusi daripada yang ditandingi parti itu pada PRU 2018.

Dalam perkembangan berkaitan, DAP dan Amanah dikatakan tidak mempunyai sebarang masalah dengan kerusi yang diberikan untuk PRN Sabah.

Sementara itu, Presiden Upko Madius Tangau enggan mengulas adakah dia berpuas hati dengan agihan kerusi tersebut. Madius kekal di majlis tersebut sehingga ia berakhir.

Partinya diberikan 12 kerusi, iaitu lebih tujuh kerusi berbanding PRU lalu.

DAP dan Amanah akan bertanding di bawah logo Warisan sementara PKR dan Upko akan menggunakan logo masing-masing.

DAP akan bertanding di tujuh kerusi, jumlah yang sama seperti yang mereka tandingi pada PRU lalu, sementara Amanah diberi satu kerusi.

Warisan akan bertanding sebanyak 54 kerusi.

Perkembangan terbaru ini mungkin mencetuskan pergeseran di antara PKR dan Warisan. PKR boleh memilih untuk bertanding di lebih banyak kerusi daripada 14 yang dituntutnya manakala Warisan juga boleh bertanding di kerusi sekutunya.

Barisan pemimpin tertinggi PKR dan Warisan hanya mempunyai dua hari sebelum hari penamaan calon pada Sabtu untuk mengurangkan pergeseran.
"""

In [22]:
string3 = """
Penubuhan universiti sukan seperti diutarakan Ketua Unit Sukan Kementerian Pengajian Tinggi, Dr Pekan Ramli dan disokong Pakar Pembangunan Sukan dan Reakreasi Luar, Universiti Pendidikan Sultan Idris (UPSI), Prof Dr Md Amin Md Taaf seperti disiarkan akhbar ini, memberikan sinar harapan kepada kewujudan institusi sedemikian.

Ia menjadi impian atlet negara untuk mengejar kejayaan dalam bidang sukan dan kecemerlangan dalam akademik untuk menjamin masa depan lebih baik apabila bersara daripada arena sukan kelak.

Pelbagai pandangan, idea, kaedah, bukti dan cadangan dilontarkan pakar berikutan pentingnya universiti sukan yang akan memberi impak besar sama ada pada peringkat kebangsaan mahupun antarabangsa.

Negara lain sudah lama meraih laba dengan kewujudan universiti sukan seperti China, Korea, Japan, Taiwan, India dan Vietnam. Mereka menghasilkan atlet universiti yang mempamerkan keputusan cemerlang pada peringkat tinggi seperti Sukan Olimpik, Kejohanan Dunia dan Sukan Asia.

Justeru, kejayaan mereka perlu dijadikan rujukan demi memajukan sukan tanah air. Jika kita merujuk pendekatan Asia, kewujudan universiti sukan penting dan memberi kesan positif dalam melonjakkan prestasi sukan lebih optimum.

Namun, jika kita melihat pendekatan Eropah, universiti sukan bukan antara organisasi atau institusi penting yang diberi perhatian dalam menyumbang kepada pemenang pingat.

Antara isu dalam universiti sukan ialah kos tinggi, lokasi, prasarana sukan, pertindihan kursus dengan universiti sedia ada dan impak terhadap dunia sukan negara hingga mengundang persoalan kewajaran dan kerelevanan penubuhannya.

Namun sebagai bekas atlet memanah negara dan Olympian (OLY) di Sukan Olimpik 2004 di Athens, Greece serta bekas pelajar Sekolah Sukan Bukit Jalil hingga berjaya dalam dunia akademik, saya mendapati terdapat beberapa faktor sering menjadi halangan dalam rutin harian mereka.

Antaranya, faktor masa yang terpaksa bergegas menghadiri kuliah selepas tamat sesi latihan yang mengambil masa 15 hingga 20 minit dengan menunggang motosikal; kereta (20-30 minit) atau pengangkutan disediakan Majlis Sukan Negara (MSN) ke Universiti Putra Malaysia (UPM).

Jika mereka menuntut di Universiti Teknologi MARA (UiTM) atau Universiti Malaya (UM), ia mungkin lebih lama.

Walaupun di universiti tersedia dengan kemudahan kolej dan kemudahan sukan, mereka memilih pulang ke MSN untuk menjalani latihan bersama pasukan dan jurulatih di padang atau gelanggang latihan rasmi.

Ini berlanjutan selagi bergelar atlet negara yang perlu memastikan prestasi sentiasa meningkat dari semasa ke semasa tanpa mengabaikan tugas sebagai pelajar.

Alangkah baiknya jika sebahagian Sekolah Sukan Bukit Jalil itu sendiri dijadikan Kolej atau Universiti Sukan Malaysia kerana lengkap dari segi kemudahan prasarana sukannya dan proses pengajaran dan pembelajaran (PdP) dalam bidang Sains Sukan, Kejurulatihan, Pendidikan Jasmani dan setaraf dengannya.

Pengambilan setiap semester pula hanya terhad kepada atlet berstatus kebangsaan dan antarabangsa sahaja supaya hasrat melahirkan lebih ramai atlet bertaraf Olimpik mudah direalisasikan.

Contohnya, bekas atlet lompat bergalah negara, Roslinda Samsu yang juga pemenang pingat perak Sukan Asia Doha 2006 dan Penerima Anugerah Khas Majlis Anugerah Sukan KPT 2012, terpaksa mengambil masa lebih kurang sembilan tahun untuk menamatkan ijazah Sarjana Muda Pendidikan Jasmani di UPM sepanjang 14 tahun terbabit dalam sukan olahraga.

Sepanjang tempoh bergelar atlet kebangsaan dan mahasiswa, beliau juga memenangi pingat Emas Sukan SEA empat siri berturut-turut pada 2005, 2007, 2009 dan 2011.

Begitu juga atlet kebangsaan seperti Leong Mun Yee (UPM); Pandalela Renong (UM); Bryan Nickson Lomas (UM); Cheng Chu Sian (UPM); Marbawi Sulaiman (UiTM) dan Norasheela Khalid (UPM).

Jika disenaraikan, mungkin lebih ramai lagi. Namun, pernah terlintas di fikiran mengapa hanya atlet dari sukan terjun yang dapat memenangi pingat di Sukan Olimpik? Bagaimana dengan atlet lain yang juga layak secara merit? Apakah kekangan atau masalah dihadapi sebagai atlet dan mahasiswa?

Adakah kewujudan universiti sukan akan memberi impak besar kepada kemajuan sukan negara? Jika dirancang dan diatur dengan cekap dan sistematik, ia perkara tidak mustahil dicapai.
"""

In [23]:
pad_sequences = tf.keras.preprocessing.sequence.pad_sequences

In [24]:
encoded = encoder.encode(f'{cleaning(string2)}') + [1]
s = pad_sequences([encoded], padding='post', maxlen = 2048)

In [25]:
%%time
l = sess.run(logits, feed_dict = {X: s, top_p: 0.0, temperature: 0.5})

CPU times: user 3min 22s, sys: 5.01 s, total: 3min 27s
Wall time: 26.6 s


In [26]:
encoder.decode(l[0].tolist())

'- Kerusi di Sabah tidak akan ditandingi oleh gabungan parti itu, tetapi akan menjadi yang tertinggi di negeri ini, menurut beberapa anggota Amanah. Keputusan mengejutkan itu berlaku setelah semua orang yang bertanding di kerusi itu meninggalkan parti itu pada pilihan raya umum lalu . Parti itu pada asalnya hanya mempunyai 54 perwakilan, tetapi akan mendapat jumlah perwakilan, sementara hanya hanya satu daripada tujuh kerusi yang bertanding.'

In [27]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'output/model.ckpt')

'output/model.ckpt'

In [28]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'top_p' in n.name
        or 'temperature' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'gradients' not in n.name
    ]
)
strings.split(',')

['top_p',
 'temperature',
 'pegasus/embeddings/word_embeddings',
 'pegasus/embeddings/position_embeddings',
 'Placeholder',
 'pegasus/encoder/layer_0/attention/self/LayerNorm/gamma',
 'pegasus/encoder/layer_0/attention/self/query/kernel',
 'pegasus/encoder/layer_0/attention/self/key/kernel',
 'pegasus/encoder/layer_0/attention/self/value/kernel',
 'pegasus/encoder/layer_0/attention/self/Softmax',
 'pegasus/encoder/layer_0/attention/self/Softmax_1',
 'pegasus/encoder/layer_0/attention/self/Softmax_2',
 'pegasus/encoder/layer_0/attention/self/Softmax_3',
 'pegasus/encoder/layer_0/attention/self/Softmax_4',
 'pegasus/encoder/layer_0/attention/output/dense/kernel',
 'pegasus/encoder/layer_0/intermediate/LayerNorm/gamma',
 'pegasus/encoder/layer_0/intermediate/dense/kernel',
 'pegasus/encoder/layer_0/intermediate/dense/bias',
 'pegasus/encoder/layer_0/output/dense/kernel',
 'pegasus/encoder/layer_0/output/dense/bias',
 'pegasus/encoder/layer_1/attention/self/LayerNorm/gamma',
 'pegasus/enco

In [29]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [30]:
freeze_graph('output', strings)

INFO:tensorflow:Restoring parameters from output/model.ckpt


INFO:tensorflow:Restoring parameters from output/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 366 variables.


INFO:tensorflow:Froze 366 variables.


INFO:tensorflow:Converted 366 variables to const ops.


INFO:tensorflow:Converted 366 variables to const ops.


24644 ops in the final graph.


In [31]:
from tensorflow.tools.graph_transforms import TransformGraph

In [32]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

In [33]:
pb = 'output/frozen_model.pb'

input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())
        
inputs = ['Placeholder', 'top_p', 'temperature']
transformed_graph_def = TransformGraph(input_graph_def, 
                                       inputs,
                                       ['logits'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


Instructions for updating:
Use tf.gfile.GFile.


In [34]:
def load_graph(frozen_graph_filename, **kwargs):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # https://github.com/onnx/tensorflow-onnx/issues/77#issuecomment-445066091
    # to fix import T5
    for node in graph_def.node:
        if node.op == 'RefSwitch':
            node.op = 'Switch'
            for index in xrange(len(node.input)):
                if 'moving_' in node.input[index]:
                    node.input[index] = node.input[index] + '/read'
        elif node.op == 'AssignSub':
            node.op = 'Sub'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'AssignAdd':
            node.op = 'Add'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
        elif node.op == 'Assign':
            node.op = 'Identity'
            if 'use_locking' in node.attr:
                del node.attr['use_locking']
            if 'validate_shape' in node.attr:
                del node.attr['validate_shape']
            if len(node.input) == 2:
                node.input[0] = node.input[1]
                del node.input[1]

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph


In [35]:
g = load_graph('output/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
top_p = g.get_tensor_by_name('import/top_p:0')
temperature = g.get_tensor_by_name('import/temperature:0')
logits = g.get_tensor_by_name('import/logits:0')
test_sess = tf.InteractiveSession(graph = g)

In [36]:
%%time
l = test_sess.run(logits, feed_dict = {x: s, top_p: 0.0, temperature: 0.0})
encoder.decode([i for i in l[0].tolist() if i > 0])

CPU times: user 7min 43s, sys: 18.6 s, total: 8min 2s
Wall time: 50.8 s


'- Kerusi di kerusi Parlimen di Sabah tidak akan ditandingi oleh Presiden PKR, Liew Junung, yang akan meninggalkan tujuh kerusi yang ada di negeri ini. Tetapi adakah ia akan kekal? "Amaran hati saya bahawa saya tidak berpuas hati dengan jumlah kerusi Warisan untuk menyerahkan kerusi itu kepada saya," kata Liew kepada Madius Tangau. "Masih ada satu kerusi yang boleh diberikan, tetapi saya tidak berpuas hati dengan jumlah kerusi yang diberikan oleh PKR. " Kerusi itu ditandingi oleh gabungan tujuh kerusi dan dewan yang ada di negeri ini . Tetapi, seperti yang dinyatakan oleh Madius Tangau, "penduduk PKR" tidak berpuas hati dengan jumlah kerusi yang diberikan oleh PKR. "Amaran itu berlaku ketika parti itu bersiap untuk meninggalkan pilihan raya umum ke-14 tetapi kemudian akan menyerahkan kerusi itu kepada pemimpin yang akan ditandingi oleh DAP," kata Madius Tangau.'

In [37]:
# g = load_graph('output/frozen_model.pb.quantized')
# x = g.get_tensor_by_name('import/Placeholder:0')
# top_p = g.get_tensor_by_name('import/top_p:0')
# temperature = g.get_tensor_by_name('import/temperature:0')
# logits = g.get_tensor_by_name('import/logits:0')
# test_sess = tf.InteractiveSession(graph = g)

In [38]:
# %%time
# l = test_sess.run(logits, feed_dict = {x: s, top_p: 0.0, temperature: 0.0})
# encoder.decode([i for i in l[0].tolist() if i > 0])